<a href="https://colab.research.google.com/github/AnnaYx/Threads/blob/main/Threads%2C_gar%C3%A7ons_e_cozinheiros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import threading
import time
import queue
import random

#classe garçom
class Garcom(threading.Thread):
    def __init__(self, nome, pedidos_pendentes, condição, max_pedidos):
        threading.Thread.__init__(self)
        self.nome = nome
        self.pedidos_pendentes = pedidos_pendentes
        self.condição = condição
        self.max_pedidos = max_pedidos
        self.pedidos_feitos = 0
        self.pedidos_entregues = 0

    def run(self):
        while self.pedidos_feitos < 10:
            pedido = self.pedidos_feitos + 1
            with self.condição:
                while len(self.pedidos_pendentes) >= self.max_pedidos:
                    print(f"{self.nome} aguardando...")
                    self.condição.wait()
                self.pedidos_pendentes.append(pedido)
                print(f"{self.nome} gerou pedido {pedido}")
                self.pedidos_feitos += 1
                self.condição.notify_all()
            time.sleep(random.uniform(0.5, 1.0))  # tempo para tomar um pedido


if __name__ == "__garcom__":
  Garcom()

#classe cozinheiro
class Cozinheiro(threading.Thread):
    def __init__(self, nome, pedidos_pendentes, condição, max_pedidos_totais, pedidos_terminados):
        threading.Thread.__init__(self)
        self.nome = nome
        self.pedidos_pendentes = pedidos_pendentes
        self.condição = condição
        self.max_pedidos_totais = max_pedidos_totais
        self.pedidos_terminados = pedidos_terminados

#fiz o laço de repetição de forma com que ficasse certo, o break saia apenas do looping interno
    def run(self):
        while True:
            with self.condição:
                while len(self.pedidos_pendentes) == 0:
                    if self.pedidos_terminados[0] == self.max_pedidos_totais:
                        print(f"{self.nome} terminou de preparar todos os pedidos")
                        return  #troquei break por return
                    else:
                        print(f"{self.nome} aguardando...")
                        self.condição.wait()
                if self.pedidos_terminados[0] == self.max_pedidos_totais: #mesmo que tenha mais pedidos, não continua ao atingir o maximo permitido
                    print(f"{self.nome} terminou de preparar todos os pedidos")
                    return #troquei
                pedido = self.pedidos_pendentes.pop(0)
                print(f"{self.nome} preparou pedido {pedido}")
                self.pedidos_terminados[0] += 1
                self.condição.notify_all()
            time.sleep(random.uniform(0.5, 1.0))


if __name__ == "__cozinheiro__":
  Cozinheiro()

def relatorio(tempo_total):
  with open("relatorio_atendimento.txt", "w") as file:
        file.write(f"Tempo total de atendimento: {tempo_total:.2f} segundos\n")
  print("Relatório gerado: relatorio_atendimento.txt")


def atendimento1():
    print("Atendimento 1: ")
    pedidos_pendentes = []
    condição = threading.Condition()
    max_pedidos = 4
    pedidos_terminados = [0]
    max_pedidos_totais = 40

    garçom1 = Garcom("Garçom 1", pedidos_pendentes, condição, max_pedidos)
    garçom2 = Garcom("Garçom 2", pedidos_pendentes, condição, max_pedidos)
    garçom3 = Garcom("Garçom 3", pedidos_pendentes, condição, max_pedidos)
    garçom4 = Garcom("Garçom 4", pedidos_pendentes, condição, max_pedidos)
    cozinheiro1 = Cozinheiro("Cozinheiro", pedidos_pendentes, condição, max_pedidos_totais, pedidos_terminados)

    tempo_inicial = time.time()

    garçom1.start()
    garçom2.start()
    garçom3.start()
    garçom4.start()
    cozinheiro1.start()

    garçom1.join() #joins garçom
    garçom2.join()
    garçom3.join()
    garçom4.join()
    cozinheiro1.join()  #join cozinheiro

    tempo_total = time.time() - tempo_inicial
    return tempo_total

def atendimento2():
    print("\nAtendimento 2:")
    pedidos_pendentes = []
    condição = threading.Condition()
    max_pedidos = 4
    pedidos_terminados = [0]
    max_pedidos_totais = 40

    garçom1 = Garcom("Garçom 1", pedidos_pendentes, condição, max_pedidos)
    garçom2 = Garcom("Garçom 2", pedidos_pendentes, condição, max_pedidos)
    garçom3 = Garcom("Garçom 3", pedidos_pendentes, condição, max_pedidos)
    garçom4 = Garcom("Garçom 4", pedidos_pendentes, condição, max_pedidos)
    cozinheiro1 = Cozinheiro("Cozinheiro 1", pedidos_pendentes, condição, max_pedidos_totais, pedidos_terminados)
    cozinheiro2 = Cozinheiro("Cozinheiro 2", pedidos_pendentes, condição, max_pedidos_totais, pedidos_terminados)

    tempo_inicial = time.time()

    garçom1.start()
    garçom2.start()
    garçom3.start()
    garçom4.start()
    cozinheiro1.start()
    cozinheiro2.start()

    garçom1.join()  # Joins garçom
    garçom2.join()
    garçom3.join()
    garçom4.join()
    cozinheiro1.join()  # Join cozinheiro
    cozinheiro2.join()  # Join segundo cozinheiro

    tempo_total = time.time() - tempo_inicial
    return tempo_total

def main():
  tempo_total1 = atendimento1()
  tempo_total2 = atendimento2()
  with open("relatorio_atendimento.txt", "w") as file:
        file.write(f"Tempo total do atendimento 1: {tempo_total1:.2f} segundos\n")
        file.write(f"Tempo total do atendimento 2: {tempo_total2:.2f} segundos\n")
        if tempo_total1 > tempo_total2:
          file.write(f"O atendimento com menor tempo foi o atendimento 2, com uma diferença de {(tempo_total1 - tempo_total2):.2f} segundos\n")
        elif tempo_total2 > tempo_total1:
          file.write(f"O atendimento com menor tempo foi o atendimento 1, com uma diferença de {(tempo_total2 - tempo_total1):.2f} segundos\n")
        else:
          file.write("Não houve diferença no tempo de atendimento.")
  print("Relatório gerado: relatorio_atendimento.txt")

if __name__ == "__main__":
    main()

Atendimento 1: 
Garçom 1 gerou pedido 1
Garçom 2 gerou pedido 1
Garçom 3 gerou pedido 1
Garçom 4 gerou pedido 1
Cozinheiro preparou pedido 1
Garçom 1 gerou pedido 2
Cozinheiro preparou pedido 1
Garçom 2 gerou pedido 2
Garçom 3 aguardando...
Garçom 4 aguardando...
Garçom 1 aguardando...
Cozinheiro preparou pedido 1
Garçom 1 gerou pedido 3
Garçom 4 aguardando...
Garçom 3 aguardando...
Garçom 2 aguardando...
Cozinheiro preparou pedido 1
Garçom 3 gerou pedido 2
Garçom 2 aguardando...
Garçom 4 aguardando...
Garçom 1 aguardando...
Garçom 3 aguardando...
Cozinheiro preparou pedido 2
Garçom 3 gerou pedido 3
Garçom 4 aguardando...
Garçom 2 aguardando...
Garçom 1 aguardando...
Cozinheiro preparou pedido 2
Garçom 4 gerou pedido 2
Garçom 2 aguardando...
Garçom 1 aguardando...
Garçom 3 aguardando...
Cozinheiro preparou pedido 3
Garçom 2 gerou pedido 3
Garçom 1 aguardando...
Garçom 3 aguardando...
Garçom 4 aguardando...
Garçom 2 aguardando...
Cozinheiro preparou pedido 2
Garçom 1 gerou pedido 4
Garç

In [ ]:
!pwd

/content
